In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ResNet50 Model 
def get_resnet50_model():
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 2)
    return model.to(device)

#Training 
def train_model(data_dir, save_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    dataset = datasets.ImageFolder(data_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = get_resnet50_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(10):
        model.train()
        total_loss = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss = {total_loss / len(loader):.4f}")

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    return model

# Open-Set Evaluation
def evaluate_open_set(model, data_dir, thresholds=np.arange(0.5, 0.96, 0.05)):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    dataset = datasets.ImageFolder(data_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)

    model.eval()
    y_true, all_probs = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            probs = torch.softmax(out, dim=1)
            y_true.extend(y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    y_true = np.array(y_true)
    all_probs = np.array(all_probs)
    y_scores = all_probs[:, 1]

    best_f1 = -1
    best_threshold = 0.5
    best_metrics = {}

    for threshold in thresholds:
        conf = np.max(all_probs, axis=1)
        pred = np.argmax(all_probs, axis=1)
        pred = np.where(conf >= threshold, pred, -1)

        known_mask = pred != -1
        y_eval = [y for y, k in zip(y_true, known_mask) if k]
        p_eval = [p for p, k in zip(pred, known_mask) if k]

        if len(y_eval) == 0 or len(p_eval) == 0:
            continue

        acc = accuracy_score(y_eval, p_eval)
        precision = precision_score(y_eval, p_eval, zero_division=0)
        recall = recall_score(y_eval, p_eval, zero_division=0)
        f1 = f1_score(y_eval, p_eval, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = round(threshold, 2)
            best_metrics = {
                'Accuracy': acc,
                'Precision': precision,
                'Recall': recall,
                'F1 Score': f1,
                'AUROC': roc_auc_score(y_true, y_scores) if len(set(y_true)) > 1 else float('nan'),
                'Rejected Unknowns': len(y_true) - len(y_eval)
            }

    best_metrics['Best Threshold'] = best_threshold
    return best_metrics

# t-SNE Visualization
def plot_tsne(model, data_dir, save_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    dataset = datasets.ImageFolder(data_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)

    model.eval()
    features, labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            feat = model.avgpool(model.layer4(model.layer3(model.layer2(model.layer1(
                model.maxpool(model.relu(model.bn1(model.conv1(x))))
            )))))
            feat = torch.flatten(feat, 1)
            features.append(feat.cpu().numpy())
            labels.extend(y.numpy())

    features = np.concatenate(features)
    tsne = TSNE(n_components=2, init='pca', learning_rate='auto', random_state=42).fit_transform(features)

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(tsne[:, 0], tsne[:, 1], c=labels, cmap='coolwarm', alpha=0.7)
    plt.legend(*scatter.legend_elements(), title="Classes")
    plt.title(f"t-SNE: {data_dir}")
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path)
    plt.close()

# Main
if __name__ == "__main__":
    train_dir = "White_augmented"
    model_save_path = "Trained_Models/ResNet50/ResNet50_Single_White.pth"
    results_csv_path = "Plots/ResNet50/ResNet50_Single.csv"
    os.makedirs(os.path.dirname(results_csv_path), exist_ok=True)

    model = train_model(train_dir, model_save_path)

    test_races = [
        "Black_augmented",
        "Indian_augmented",
        "East_Asian_augmented",
        "Southeast_Asian_augmented",
        "Latino_Hispanic_augmented"
    ]

    all_results = []
    for test_dir in test_races:
        print(f"\nEvaluating on: {test_dir}")
        metrics = evaluate_open_set(model, test_dir)
        plot_tsne(model, test_dir, f"Plots/ResNet50/tsne_ResNet50_Single_{test_dir}.png")

        flat_metrics = {
            'Race': test_dir,
            'Accuracy': metrics['Accuracy'],
            'Precision': metrics['Precision'],
            'Recall': metrics['Recall'],
            'F1 Score': metrics['F1 Score'],
            'AUROC': metrics['AUROC'],
            'Best Threshold': metrics['Best Threshold'],
            'Rejected Unknowns': metrics['Rejected Unknowns']
        }
        all_results.append(flat_metrics)

    df = pd.DataFrame(all_results)
    df.to_csv(results_csv_path, index=False)

Epoch 1: Loss = 0.6465
Epoch 2: Loss = 0.5920
Epoch 3: Loss = 0.5280
Epoch 4: Loss = 0.4875
Epoch 5: Loss = 0.4750
Epoch 6: Loss = 0.4055
Epoch 7: Loss = 0.3835
Epoch 8: Loss = 0.3319
Epoch 9: Loss = 0.3294
Epoch 10: Loss = 0.2700

Evaluating on: Black_augmented

Evaluating on: Indian_augmented

Evaluating on: East_Asian_augmented

Evaluating on: Southeast_Asian_augmented

Evaluating on: Latino_Hispanic_augmented
